In [139]:
import csv
import matplotlib.pyplot as mp
import numpy as np
import random as rand
import nbimporter
from importlib import reload
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import data
reload(data)

Using matplotlib backend: Qt5Agg


<module 'data' from 'C:\\Users\\sebastien\\google drive\\PanAcces\\projects\\projet3\\data.py'>

In [57]:
full_data=data.data_set("handwriting.csv","fatigue.csv")
full_data.divide("training_set.csv","validation_set.csv")
training_set=data.data_set("training_set.csv","fatigue.csv")
validation_set=data.data_set("validation_set.csv","fatigue.csv")

removing subject  76


In [153]:
t_labels,t_data=training_set.get_set("pain")
t_data[0].shape

(6,)

In [154]:
model = keras.Sequential([
    keras.layers.Dense(6,activation=tf.nn.tanh),
    keras.layers.Dense(12, activation=tf.nn.tanh),
    keras.layers.Dense(24, activation=tf.nn.tanh),
    keras.layers.Dense(6, activation=tf.nn.softmax)
])
model.compile(optimizer=keras.optimizers.Nadam(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [156]:
model.fit(t_data, t_labels, epochs=10)

Epoch 1/10
1604/1604 [==============================] - 12s 7ms/sample - loss: 1.0004 - acc: 0.7363
Epoch 2/10
1604/1604 [==============================] - 0s 192us/sample - loss: 0.7620 - acc: 0.8155
Epoch 3/10
1604/1604 [==============================] - 0s 202us/sample - loss: 0.7608 - acc: 0.8155
Epoch 4/10
1604/1604 [==============================] - 0s 231us/sample - loss: 0.7596 - acc: 0.8155
Epoch 5/10
1604/1604 [==============================] - 1s 313us/sample - loss: 0.7609 - acc: 0.8155
Epoch 6/10
1604/1604 [==============================] - 0s 291us/sample - loss: 0.7597 - acc: 0.8155
Epoch 7/10
1604/1604 [==============================] - 0s 173us/sample - loss: 0.7581 - acc: 0.8155
Epoch 8/10
1604/1604 [==============================] - 0s 170us/sample - loss: 0.7594 - acc: 0.8155
Epoch 9/10
1604/1604 [==============================] - 0s 202us/sample - loss: 0.7586 - acc: 0.8155
Epoch 10/10
1604/1604 [==============================] - 0s 223us/sample - loss: 0.7597 - ac

In [138]:
v_labels,v_data=validation_set.get_set("pain")
test_loss, test_acc = model.evaluate(v_data, v_labels)

407/407 [==============================] - 1s 2ms/sample - loss: 108.5927 - mean_absolute_error: 9.9697 - mean_squared_error: 108.5927


ValueError: too many values to unpack (expected 2)

In [121]:
test_loss

2.2863519718957472

In [122]:
test_acc

0.17199017

In [123]:
predictions = model.predict(v_data)
predictions.shape
print(predictions[22])
print(max(predictions[22]))
predictions[22][0]

[1.7827898e-06 4.4355470e-06 3.7438492e-06 3.2718742e-06 1.5970681e-02
 7.1125948e-03 2.2841388e-02 1.1632961e-01 9.5333382e-02 1.4921412e-01
 1.6127416e-01 1.5657990e-01 7.9317853e-02 4.4558141e-02 3.5437558e-02
 1.5249287e-06 8.0839410e-02 3.5171047e-02 2.3822331e-06 3.0127114e-06]
0.16127416


1.7827898e-06

In [124]:
print(v_labels)

[10 10 10 10  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9 12 12 12
 12 16 16 16 16 16 16  8  8  8  8  7  7  7  6  6  6  6  6  7  7  7  7  7
 12 12 12 12 12  9  9  9  9  9  9  9  9  9  9  8  8  8  8 12 12 12 12 12
 12  9  9  9  9  9  9  5  5  5  5  5  5  5 16 16 16 16 16 16 16  9  9  9
  9  9  9  9  9  9 13 13 13 13 13  7  7  7  7  7  7  9  9  9  9  9  9 10
 10  7  7  7  7  7  7  7 16 16 16 16 12 12 12 12 12 12 12  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8 17 17 17 17 11 11 11 11 11 11 11 11
 11 11 11  6  6  6  6  6  6  6 11 11 11 11  4  4  4  4  4  4 14 14 14 11
 11 11 11 11 11 11 11 11 11 11 11 11 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 14 14 14 14 14 14  4  4  4  4  4 11 11 11 11 11 11 11
  8  8  8  8  8  7  7  7  7  7 16 16 16 16 16 16 16 10 10 10 10 10 10  7
  7  7 13 13 13 13 13 13  9  9  9 11 11 11 11 11 11 11 12 12 12 12 12 12
 12 10 10 10 10 10 10 10  6  6  6  6  6  6  6  6  6  4  4  4  4  4 13 13
 13 13 13 13 13 13 13 13 12 12 12 12 12 11 11 11 11